In [ ]:
# .)RNN is well when we hve to deal with sequence of data and LSTM is its type so it 
# deals perfectly with sequence of data .
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# Time Series Prediction using LSTM (vanilla Recurrent Neural Network)
# Long short-term memory (LSTM) is an artificial recurrent neural network (RNN)
# architecture used in the field of deep learning. Unlike standard feedforward neural
# networks, LSTM has feedback connections.

# LSTM are meant to selectively capture long and short term dependencies in a sequence.

# Problems of traditional regression based forecasting models

# Do not support

# 1. noise, missing data or outliers.

# 2. non-linear relationship.

# 3. mutiple fields to influnce the predictions.

# Neural networks add the capability to learn possibly noisy and nonlinear relationships 
# with arbitrarily defined but fixed numbers of inputs and outputs supporting multivariate 
# and multi-step forecasting.

In [ ]:
# dataset linbk 
# https://github.com/plotly/datasets/blob/master/monthly-milk-production-pounds.csv

In [ ]:
df = pd.read_csv('./data/monthly_milk_production.csv',index_col='Date',parse_dates=True)
# .)the blw MS means it will now show the data monthly wise . 
df.index.freq='MS'

In [ ]:
df.head()

In [ ]:
df.plot(figsize=(8,8))

In [ ]:
# .)the above graph was incresng with time to see in more detai about its seasonal trend 
# what its is trend nature we use the blw fun seasonal_decompose. 
from statsmodels.tsa.seasonal import seasonal_decompose

In [ ]:
results = seasonal_decompose(df['Production'])
results.plot()

In [ ]:
# .)The ARIMA models are regression models  in order to use that we hve to firstly 
# convert the data into stationary data but in RNN LSTM it deals with sequence of data
# so no need to do like this conversion into stationary data we can do it directly.
# if model doent perform battern then in RNN we can use ststionary data . 

In [ ]:
len(df)

In [ ]:
train = df.iloc[:156]
test = df.iloc[156:]

In [ ]:
# good step to preproces in min max.
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

In [ ]:
df.head(),df.tail()

In [ ]:
scaler.fit(train)
scaled_train = scaler.transform(train)
scaled_test = scaler.transform(test)

In [ ]:
from keras.preprocessing.sequence import TimeseriesGenerator
# define generator
n_input = 3
n_features = 1
generator = TimeseriesGenerator(scaled_train, scaled_train, length=n_input, batch_size=1)

In [ ]:

X,y = generator[0]
print(f'Given the Array: \n{X.flatten()}')
print(f'Predict this y: \n {y}')

In [ ]:
# .)First 1 is the no of rows and 3 is input features and then 1 is no of features or output
X.shape

In [ ]:
# We do the same thing, but now instead for 12 months
n_input = 12
generator = TimeseriesGenerator(scaled_train, scaled_train, length=n_input, batch_size=1)

In [ ]:
X,y = generator[0]
print(f'Given the Array: \n{X.flatten()}')
print(f'Predict this y: \n {y}')

In [ ]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM

In [ ]:
# define model
model = Sequential()
# .)Blw 100 is LSTM neurons.
model.add(LSTM(100, activation='relu', input_shape=(n_input, n_features)))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')

In [ ]:
model.summary()

In [ ]:
# fit model
model.fit(generator,epochs=50)

In [ ]:
loss_per_epoch = model.history.history['loss']
plt.plot(range(len(loss_per_epoch)),loss_per_epoch)

In [ ]:
# Tking last 12 month value and predicting the test[0] value . 
last_train_batch = scaled_train[-12:]

In [ ]:
last_train_batch = last_train_batch.reshape((1, n_input, n_features))

In [ ]:
model.predict(last_train_batch)

In [ ]:
scaled_test[0]

In [ ]:
test_predictions = []
first_eval_batch = scaled_train[-n_input:]
current_batch = first_eval_batch.reshape((1, n_input, n_features))

for i in range(len(test)):
    
    # get the prediction value for the first batch
    current_pred = model.predict(current_batch)[0]
    
    # append the prediction into the array
    test_predictions.append(current_pred) 
    
    # use the prediction to update the batch and remove the first value
    current_batch = np.append(current_batch[:,1:,:],[[current_pred]],axis=1)

In [ ]:
test_predictions

In [ ]:
test.head()

In [ ]:
true_predictions = scaler.inverse_transform(test_predictions)

In [ ]:
test['Predictions'] = true_predictions

In [ ]:
test.head()

In [ ]:
import matplotlib.pyplot as plt 
plt.plot(test['Production'].values,label="actual")
plt.plot(test['Predictions'].values,label="predictions")
plt.legend()
plt.show()